<hr>
# [cknowledge.org](http://cknowledge.org): Community-driven benchmarking and optimization of computing systems - from classical to quantum
<hr>

[Quantum Computing](https://github.com/ctuning/ck-quantum/wiki)
* [CK-QISKit](https://github.com/ctuning/ck-qiskit) (IBM)
* [CK-Rigetti](https://github.com/ctuning/ck-rigetti) ([Rigetti Computing](https://rigetti.com/))
* [CK-ProjectQ](https://github.com/ctuning/ck-projectq) ([ProjectQ](https://projectq.ch/))

[Artificial Intelligence and Machine Learning](http://cknowledge.org/ai)
* [Reproducible Quality-Efficient Systems Tournaments](http://cknowledge.org/request) ([ReQuEST initiative](http://cknowledge.org/request.html#organizers))
* [AI artifacts](http://cknowledge.org/ai-artifacts) (cTuning foundation)
* [Android app](https://play.google.com/store/apps/details?id=openscience.crowdsource.video.experiments) (dividiti)
* [Desktop app](https://github.com/dividiti/ck-crowdsource-dnn-optimization) (dividiti)
* [CK-Caffe](https://github.com/dividiti/ck-caffe) (Berkeley)
* [CK-Caffe2](https://github.com/ctuning/ck-caffe2) (Facebook)
* [CK-CNTK](https://github.com/ctuning/ck-cntk) (Microsoft)
* [CK-KaNN](https://github.com/dividiti/ck-kann) (Kalray)
* [CK-MVNC](https://github.com/ctuning/ck-mvnc) (Movidius / Intel)
* [CK-MXNet](https://github.com/ctuning/ck-mxnet) (Apache)
* [CK-NNTest](https://github.com/ctuning/ck-nntest) (cTuning foundation)
* [CK-TensorFlow](https://github.com/ctuning/ck-tensorflow) (Google)
* [CK-TensorRT](https://github.com/dividiti/ck-tensorrt) (NVIDIA)
* etc.

<hr>
# Variational Quantum Eigensolver on Rigetti
[Quantum Collective Knowledge Hackaton](https://www.eventbrite.co.uk/e/quantum-computing-hackathon-tickets-46441126660#), [Centre for Mathematical Science - University of Cambridge](http://www.cms.cam.ac.uk/), 15 June 2018

## Table of Contents

1. [Organisers](#organisers)
1. [Overview](#overview)
1. [Metric](#metric)
1. [Setting up](#settingup)
1. [Running experiments](#running)
1. [Experimental data](#data)
1. [Data wrangling code](#code) (for developers)
1. [Analysis](#analysis)

<a id="organisers"></a>
## Organisers

- [Rigetti Computing](https://rigetti.com): access to [Quantum Virtual Machine](http://pyquil.readthedocs.io/en/latest/qvm.html) (QVM) and [Quantum Processing Unit](http://pyquil.readthedocs.io/en/latest/qpu.html) (QPU).
- [River Lane Research](https://riverlane.io/): Steve Brierley, Oscar Higgott, Daochen Wang, Amy Flower
- [dividiti](http://dividiti.com/): Anton Lokhmotov, Leo Gordon, Flavio Vella, Grigori Fursin

<a id="overview"></a>
## Overview

This Jupyter Notebook studies the performance of [Variational-Quantum-Eigensolver](http://grove-docs.readthedocs.io/en/latest/vqe.html) (VQE) on [Rigetti](https://rigetti.com/)'s platforms.

<a id="metric"></a>
## Time-to-solution metric

To compare solutions of participants we use a **time-to-solution** $T$ metric defined as follows.

### Definition

Let's assume that to find the ground state of a given molecule (e.g. [Helium](https://en.wikipedia.org/wiki/Helium)) a participant makes $N$ runs of their implementation (e.g. $N=3$). A run is considered _successful_ if the ground state found in this run is equal to the ground state known for the molecule to given precision $\delta$ (e.g. $\delta=0.1$). Assume that a single run takes $t$ samples (calls to the quantum computer) on average.

Let $s$ be the probability of success of the participant's VQE implementation (i.e. the number of successful runs divided by the total number of runs $N$).

Let $R$ be the number of runs required to find the ground state with given probability $p$ (e.g. $p=0.6$):
\begin{equation}
R = {\frac{\log(1-p)}{\log(1-s)}}.
\end{equation}

The **time-to-solution** $T$, defined as the total number of samples used throughout the whole optimisation procedure of VQE, is then calculated as:
\begin{equation}
T = R \times t.
\end{equation}

### Derivation

If the probability of success is $s$, then the probability of _failing to find_ the ground state after $R$ runs is $(1-s)^R$. Therefore, the probability of finding the ground state at least once after $R$ runs is $p =1 - (1-s)^R$. Therefore, the number of runs $R$ required to find the ground state at least once with probability $p$ can be found by solving $p=1-(1-s)^R$.

### Uncertainties

**TBD**

<!--

We can also calculate the standard error associated with the calculated time-to-solution $T$. 

From the [binomial distribution](Binomial_distribution), the uncertainty $\sigma_s$ in the success probability $s$ is:
\begin{equation}
  \sigma_s=\sqrt{\frac{s(1-s)}{N}}
\end{equation}
where $N$ is the number of runs used to determine $s$.

The uncertainty in the time taken per run $t$ is:
\begin{equation}
  \sigma_t=\frac{\mathrm{std}}{\sqrt{N}}
\end{equation}
where $\mathrm{std}$ is the standard deviation of the times taken by all $N$ runs.

The uncertainty in total time taken is:
\begin{equation}
\sigma_T=\sqrt{0.25 \cdot (T(t+\sigma_t, s) - T(t-\sigma_t,s))^2 + (T(t, s + \sigma_s) - T(t,s))^2}
\end{equation}

!-->

<a id="settingup"></a>
## Setting up

Please follow instructions [here](https://github.com/ctuning/ck-quantum/blob/master/README.md).

<a id="running"></a>
## Running experiments

```
$ ck benchmark program:rigetti-vqe \
  --env.RIGETTI_QUANTUM_DEVICE=<platform> \
  --env.VQE_MINIMIZER_METHOD=<minimizer_method> \
  --env.VQE_SAMPLE_SIZE=<sample_number> \
  --env.VQE_MAX_ITERATIONS=<max_iterations> \
  --record --record_repo=local --record_uoa=<email>-<plaform> \
  --tags=qck,hackathon-2018_06_15,<email>,<platform>,<minimizer_method> \
  --repetitions=<repetitions>
```
where:
- `platform`: `8Q-Agave` or `QVM`;
- `minimizer_method`: `my_melder_nead` or `my_cobyla` or `my_minimizer` (as defined in [optimizers.py](https://github.com/ctuning/ck-quantum/blob/master/package/tool-hackathon/hackathon-src/hackathon/optimizers.py) installed under e.g. `$CK_TOOLS/hackathon-1.0-linux-64/lib/hackathon`);
- `sample_size`: e.g. `100` (or another resolution);
- `max_iterations`: e.g. `80` (or another cut-off point);
- `email`: a valid email address (later to be replaced with a team id e.g. `team01`);
- `repetitions`: how many times to run the experiment with the given parameters: e.g. `3`.

<a id="data"></a>
## Get sample experimental data

The sample experimental data can be downloaded and registered with CK as follows:

```
$ wget https://www.dropbox.com/s/fvichboqnjcxc8v/ck-quantum-hackathon-sample.zip
$ ck add repo --zip=ck-quantum-hackathon-sample.zip
```

In [ ]:
repo_uoa = 'ck-quantum-hackathon-20180615-teams'
!ck list $repo_uoa:experiment:* --print_full | sort

<a id="code"></a>
## Data wrangling code

**NB:** Please ignore this section if you are not interested in re-running or modifying this notebook.

### Includes

#### Standard

In [ ]:
import os
import sys
import json
import re

#### Scientific

If some of the scientific packages are missing, please install them using:
```
# pip install jupyter pandas numpy matplotlib
```

In [ ]:
import IPython as ip
import pandas as pd
import numpy as np
import matplotlib as mp

In [ ]:
print ('IPython version: %s' % ip.__version__)
print ('Pandas version: %s' % pd.__version__)
print ('NumPy version: %s' % np.__version__)
print ('Matplotlib version: %s' % mp.__version__)

In [ ]:
from IPython.display import Image, display
def display_in_full(df):
    pd.options.display.max_columns = len(df.columns)
    pd.options.display.max_rows = len(df.index)
    display(df)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [ ]:
default_colormap = cm.autumn
default_fontsize = 16
default_barwidth = 0.8
default_figwidth = 16
default_figheight = 8
default_figdpi = 200
default_figsize = [default_figwidth, default_figheight]

In [ ]:
if mp.__version__[0]=='2': mp.style.use('classic')
mp.rcParams['figure.max_open_warning'] = 200
mp.rcParams['figure.dpi'] = default_figdpi
mp.rcParams['font.size'] = default_fontsize
mp.rcParams['legend.fontsize'] = 'medium'

#### Collective Knowledge

If CK is not installed, please install it using:
```
# pip install ck
```

In [ ]:
import ck.kernel as ck
print ('CK version: %s' % ck.__version__)

In [ ]:
# NB: Make sure the quantum hackathon tool is installed. (It should be if you have run any experiments.)
# $ ck install package --tags=ck-quantum,tool,hackathon,v1
r=ck.access({'action':'show', 'module_uoa':'env', 'tags':'tool,hackathon'})
if r['return']>0:
    print ("Error: %s" % r['error'])
    exit(1)
    
# Get the path to the first returned environment entry.
tool_hackathon_dir=r['lst'][0]['meta']['env']['CK_ENV_LIB_HACKATHON_LIB']
sys.path.append(tool_hackathon_dir)
from hackathon.utils import *

### Access experimental data

In [ ]:
def get_experimental_results(repo_uoa, tags='qck', module_uoa='experiment'):
    r = ck.access({'action':'search', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'tags':tags})
    if r['return']>0:
        print('Error: %s' % r['error'])
        exit(1)
    experiments = r['lst']
    
    dfs = []
    for experiment in experiments:
        data_uoa = experiment['data_uoa']
        r = ck.access({'action':'list_points', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'data_uoa':data_uoa})
        if r['return']>0:
            print('Error: %s' % r['error'])
            exit(1)
        tags = r['dict']['tags']

        skip = False
        # Get team name (final data) or email (submission data).
        team_tags = [ tag for tag in tags if tag.startswith('team-') ]
        email_tags = [ tag for tag in tags if tag.find('@')!=-1 ]
        if len(team_tags) > 0:
            team = team_tags[0][0:7]
        elif len(email_tags) > 0:
            team = email_tags[0]
        else:
            print('[Warning] Cannot determine team name for experiment in: \'%s\'' % r['path'])
            team = 'team-default'

        if skip:
            print('[Warning] Skipping experiment with bad tags:')
            print(tags)
            continue
    
        # For each point.    
        for point in r['points']:
            point_file_path = os.path.join(r['path'], 'ckp-%s.0001.json' % point)
            with open(point_file_path) as point_file:
                point_data_raw = json.load(point_file)
            characteristics_list = point_data_raw['characteristics_list']
            num_repetitions = len(characteristics_list)
            data = [
                {
                    # features
                    'platform': characteristics['run'].get('vqe_input', {}).get('q_device_name', 'unknown').lower(),
                    # choices
                    'minimizer_method': characteristics['run'].get('vqe_input', {}).get('minimizer_method', 'n/a'),
                    'minimizer_options': characteristics['run'].get('vqe_input', {}).get('minimizer_options', {'maxfev':-1}),
                    'minimizer_src': characteristics['run'].get('vqe_input', {}).get('minimizer_src', ''),
                    'sample_number': characteristics['run'].get('vqe_input', {}).get('sample_number','n/a'),
                    # statistical repetition
                    'repetition_id': repetition_id,
                    # runtime characteristics
                    'run': characteristics['run'],
                    'report': characteristics['run'].get('report', {}),
                    'vqe_output': characteristics['run'].get('vqe_output', {}),
                }
                for (repetition_id, characteristics) in zip(range(num_repetitions), characteristics_list)
                if len(characteristics['run']) > 0
            ]
            
            for datum in data:
                datum['team'] = team
                datum['point'] = point
                datum['success'] = datum.get('vqe_output',{}).get('success',False)
                datum['nfev'] = np.int64(datum.get('vqe_output',{}).get('nfev',-1))
                datum['nit'] = np.int64(datum.get('vqe_output',{}).get('nit',-1))
                datum['fun'] = np.float64(datum.get('vqe_output',{}).get('fun',0))
                datum['fun_validated'] = np.float64(datum.get('vqe_output',{}).get('fun_validated',0))
                datum['fun_exact'] = np.float64(datum.get('vqe_output',{}).get('fun_exact',0))
                datum['total_seconds'] = np.float64(datum.get('report',{}).get('total_seconds',0))
                datum['total_q_seconds'] = np.float64(datum.get('report',{}).get('total_q_seconds',0))
                datum['total_q_shots'] = np.int64(datum.get('report',{}).get('total_q_shots',0))
                tmp_max_iterations = list(datum.get('minimizer_options',{'maxfev':-1}).values())
                datum['max_iterations'] = tmp_max_iterations[0] if len(tmp_max_iterations)>0 else -1
            index = [
                'platform', 'team', 'minimizer_method', 'sample_number', 'max_iterations', 'point', 'repetition_id'
            ]
            # Construct a DataFrame.
            df = pd.DataFrame(data)
            df = df.set_index(index)
            # Append to the list of similarly constructed DataFrames.
            dfs.append(df)
    if dfs:
        # Concatenate all thus constructed DataFrames (i.e. stack on top of each other).
        result = pd.concat(dfs)
        result.sort_index(ascending=True, inplace=True)
    else:
        # Construct a dummy DataFrame the success status of which can be safely checked.
        result = pd.DataFrame(columns=['success'])
    return result

In [ ]:
# Merge experimental results from the same team with the same parameters
# (minimizer_method, sample_number, max_iterations) and minimizer source.
def merge_experimental_results(df):
    dfs = []
    df_prev = None
    for index, row in df.iterrows():
        # Construct a DataFrame.
        df_curr = pd.DataFrame(row).T
        # Check if this row is similar to the previous row.
        if df_prev is not None: # if not the very first row
            if df_prev.index.levels[:-2]==df_curr.index.levels[:-2]: # if the indices match for all but the last two levels
                if df_prev.index.levels[-2]!=df_curr.index.levels[-2]: # if the experiments are different
                    if df_prev['minimizer_src'].values==df_curr['minimizer_src'].values: # if the minimizer source is the same
                        print('[Info] Merging experiment:')
                        print(df_curr.index.levels)
                        print('[Info] into:')
                        print(df_prev.index.levels)
                        print('[Info] as:')
    #                     df_curr.index = df_prev.index.copy() # TODO: increment repetition_id
                        df_curr.index = pd.MultiIndex.from_tuples([(x[0],x[1],x[2],x[3],x[4],x[5],x[6]+1) for x in df_prev.index])
                        print(df_curr.index.levels)
                        print
                    else:
                        print('[Warning] Cannot merge experiments as the minimizer source is different:')
    #                     print('------------------------------------------------------------------------')
                        print(df_prev.index.levels)
    #                     print(df_prev['minimizer_src'].values[0])
    #                     print
    #                     print('------------------------------------------------------------------------')
                        print(df_curr.index.levels)
    #                     print(df_curr['minimizer_src'].values[0])
                        print
    #             else:
    #                 print('[Info] Keeping experiments separate:')
    #                 print(df_prev.index.levels)
    #                 print(df_curr.index.levels)
    #                 print
        # Append to the list of similarly constructed DataFrames.
        dfs.append(df_curr)
        # Prepare for next iteration.
        df_prev = df_curr

    # Concatenate all thus constructed DataFrames (i.e. stack on top of each other).
    result = pd.concat(dfs)
    result.index.names = df.index.names
    result.sort_index(ascending=True, inplace=True)
    
    return result

In [ ]:
def get_metrics(df, delta=0.1, prob=0.5, which_fun_key='fun_exact', which_time_key='total_q_shots'):
    dfs = []
    names_no_repetitions = df.index.names[:-1]
    for index, group in df.groupby(level=names_no_repetitions):
        # Compute metrics.
        classical_energy, minimizer_method, minimizer_src, n_succ, T_ave, T_err, t_ave, t_err, s, s_err = \
            benchmark_list_of_runs(group['run'], verbose=False, delta=delta, prob=prob,
                                   which_fun_key=which_fun_key, which_time_key=which_time_key)
        # Construct a DataFrame from the metrics.
        data = {
            # Time to solution.
            'T_ave' : T_ave,
            'T_err' : T_err,
            # Time metric (seconds or shots).
            't_ave' : t_ave,
            't_err' : t_err,
            # Tries metric.
            's' : s,
            's_err' : s_err
        }
        data.update({ k : v for (k, v) in zip(names_no_repetitions, index) })
        data['num_repetitions'] = len(group)
        # NB: index must be something.
        df_ = pd.DataFrame(data=data, index=[0])
        df_ = df_.set_index(names_no_repetitions)
        # Append to the list of similarly constructed DataFrames.
        dfs.append(df_)
    if dfs:
        # Concatenate all thus constructed DataFrames (i.e. stack on top of each other).
        result = pd.concat(dfs).dropna()
        result.sort_index(ascending=True, inplace=True)
    return result

### Plot experimental data

In [ ]:
def plot(df, platform_set=None, minimizer_method_set=None, sample_number_set=None, max_iterations_set=None,
         markersize_divisor=20,
         xmin=0.0, xmax=85.01, xstep=5.00,
         ymin=-3.0, ymax=-0.49, ystep=0.25,
         figsize=(18,9), dpi=200, legend_loc='lower right'):
    
    platform_set = platform_set or df.index.get_level_values(level='platform').unique()
    minimizer_method_set = minimizer_method_set or df.index.get_level_values(level='minimizer_method').unique()
    sample_number_set = sample_number_set or df.index.get_level_values(level='sample_number').unique()
    max_iterations_set = max_iterations_set or df.index.get_level_values(level='max_iterations').unique()

    # Options.
    minimizer_method_to_color = {
        'my_cobyla' : 'orange',
        'my_nelder_mead' : 'green',
        'my_minimizer' : 'blue'
    }
    platform_to_marker = {
        '8q-agave' : '8', # octagon
        'qvm' : 's',      # square
        'local_qasm_simulator' : 'p' # pentagon
    }
    
    last_marker_size = 10
    last_marker_color = 'black'
    last_marker_success_true = '^'
    last_marker_success_false = 'v'

    fig = plt.figure(figsize=figsize, dpi=dpi)
    ax = fig.gca()
    for index, row in df.iterrows():
        (platform, team, minimizer_method, sample_number, max_iterations, point, repetition_id) = index
        if platform not in platform_set: continue
        if sample_number not in sample_number_set: continue
        if minimizer_method not in minimizer_method_set: continue
        # NB: This uses 'fun', not 'fun_exact' or 'fun_validated'.
        energies = [ iteration['energy'] for iteration in row['report']['iterations'] ]
        marker=platform_to_marker[platform]
        markersize=sample_number/markersize_divisor
        color=minimizer_method_to_color.get(minimizer_method, 'red')
        markerfacecolor=color
        linestyle='-'
        ax.plot(range(len(energies)), energies, marker=marker, color=color, linestyle=linestyle,
                markerfacecolor=markerfacecolor, markersize=markersize)
        # Mark last function evaluation.
        last_energy = energies[-1]
        last_fev = row['nfev']-1 if minimizer_method=='my_cobyla' or 'my_nelder_mead' else row['nfev']
        last_marker = last_marker_success_true if row['success'] else last_marker_success_false
        ax.plot(last_fev, last_energy, color=last_marker_color, marker=last_marker, markersize=last_marker_size)

    # Horizontal line for the known ground state.
    plt.axhline(y=-2.80778395754, color='red', linestyle='--')
    # Vertical lines for max_iterations.
    for max_iterations in max_iterations_set:
        plt.axvline(x=max_iterations, color='black')
    # Grid.
    plt.grid()
    # Title.
    title = 'Variational Quantum Eigensolver (VQE)'
    ax.set_title(title)
    # X axis.
    xlabel='Function evaluation'
    ax.set_xlabel(xlabel)
    ax.set_xlim(xmin, xmax)
    ax.set_xticks(np.arange(xmin, xmax, xstep))
    # Y axis.
    ylabel='Energy'
    ax.set_ylabel(ylabel)
    ax.set_ylim(ymin, ymax)
    ax.set_yticks(np.arange(ymin, ymax, ystep))
    # Legend. https://matplotlib.org/users/legend_guide.html
    handles = [
        mp.lines.Line2D([], [], label='platform="%s",minimizer_method="%s"' % (p,m), color=minimizer_method_to_color.get(m, 'red'),
                        marker=platform_to_marker[p], markersize=last_marker_size)
        for p in sorted(platform_set)
        for m in sorted(minimizer_method_set)
    ]
    handles.append(mp.lines.Line2D([],[], label='ground state', color='red', linestyle='--'))
    plt.legend(handles=handles, title='platform,minimizer_method', loc=legend_loc)
    # Save figure.
#    plt.savefig('vqe.energy.png')

In [ ]:
def plot_metric(df, metric='total_q_seconds'):
    df.columns.name='metric'
    # "df.index.names[:-1]" means reduce along 'repetition_id' (statistical variation).
    df_mean = df[[metric]].groupby(level=df.index.names[:-1]).mean().unstack('platform')
    df_std = df[[metric]].groupby(level=df.index.names[:-1]).std().unstack('platform')
    ax = df_mean.plot(kind='bar', yerr=df_std, grid=True, legend=True, rot=45,
                      fontsize=default_fontsize, figsize=default_figsize, colormap=default_colormap)

<a id="analysis"></a>
## Analysis

### All experimental data

In [ ]:
df = get_experimental_results(repo_uoa=repo_uoa)
display_in_full(df)

### Merge experimental results

In [ ]:
df_merged = merge_experimental_results(df)
display_in_full(df_merged)

### Compute the time-to-solution metric etc.

In [ ]:
df_metrics = get_metrics(df_merged, delta=0.1, prob=0.5, which_fun_key='fun_exact', which_time_key='total_q_shots')
df_metrics

### The unexpected winner

**TODO:** Explain why using `sample_number=1` can unexpectedly lead to the best metric value, even on real hardware.

In [ ]:
idxmin1 = df_metrics['T_ave'].idxmin()
idxmin1

In [ ]:
df_metrics.loc[[idxmin1]]

In [ ]:
df.loc[idxmin1]

In [ ]:
# Platform, Team, minimizer Function, number of Samples, number of function eValuations, Experiment, Repetition
(p,t,f,s,v,e) = idxmin1
# Plot the winner.
plot(df.loc[[(p,t,f,s,v,e,k) for k in range(len(df.loc[idxmin1]))]],
     xmax=7, xstep=1, ymin=-3.00, ymax=0.00+0.01, legend_loc='upper center')

In [ ]:
# Exclude the winner.
df_metrics = df_metrics.drop(idxmin1)

### The conditional runner-up

**NB:** The runner-up also uses `sample_number=1` but only a single run (hence, it's a "conditional" runner-up, as we can determine the error only from multiple runs).

In [ ]:
idxmin2 = df_metrics['T_ave'].idxmin()
idxmin2

In [ ]:
df_metrics.loc[[idxmin2]]

In [ ]:
df.loc[idxmin2]

In [ ]:
# Platform, Team, minimizer Function, number of Samples, number of function eValuations, Experiment, Repetition
(p,t,f,s,v,e) = idxmin2
# Plot the winner.
plot(df.loc[[(p,t,f,s,v,e,k) for k in range(len(df.loc[idxmin2]))]],
     xmax=7, xstep=1, ymin=-3.00, ymax=0.00+0.01, legend_loc='upper center')

In [ ]:
# Print the minimizer source.
print(df.loc[(p,t,f,s,v,e,0)]['minimizer_src'])

In [ ]:
# Exclude the conditional runner-up.
df_metrics = df_metrics.drop(idxmin2)

### The QVM runner-up

In [ ]:
platform_qvm = 'qvm'
df_metrics_qvm = df_metrics.loc[platform_qvm]
idxmin_qvm = df_metrics_qvm['T_ave'].idxmin()
idxmin_qvm

In [ ]:
df_metrics.loc[platform_qvm].loc[[idxmin_qvm]]

In [ ]:
df.loc[platform_qvm].loc[idxmin_qvm]

In [ ]:
# Platform, Team, minimizer Function, number of Samples, number of function eValuations, Experiment, Repetition
p = platform_qvm
(t,f,s,v,e) = idxmin_qvm
# Plot the runner up.
plot(df.loc[[(p,t,f,s,v,e,k) for k in range(len(df.loc[(p,t,f,s,v,e)]))]],
     xmax=9, xstep=1, ymin=-3.00, ymax=0.00+0.01, legend_loc='upper right')

In [ ]:
# Print the minimizer source.
print(df.loc[(p,t,f,s,v,e,0)]['minimizer_src'])

In [ ]:
# Exclude the QVM runner-up.
df_metrics = df_metrics.drop((p,t,f,s,v,e))

### The QPU runner-up

In [ ]:
platform_qpu = '8q-agave'
df_metrics_qpu = df_metrics.loc[platform_qpu]
idxmin_qpu = df_metrics_qpu['T_ave'].idxmin()
idxmin_qpu

In [ ]:
df_metrics_qpu.loc[[idxmin_qpu]]

In [ ]:
df.loc[platform_qpu].loc[idxmin_qpu]

In [ ]:
# Platform, Team, Experiment, minimizer Function, number of Samples, number of function eValuations, Repetition
p = platform_qpu
(t,f,s,v,e) = idxmin_qpu
# Plot the QPU runner-up.
plot(df.loc[[(p,t,f,s,v,e,k) for k in range(len(df.loc[(p,t,f,s,v,e)]))]],
     xmax=7, xstep=1, ymin=-3.00, ymax=-0.00+0.01, legend_loc='upper right')

In [ ]:
# Print the minimizer source.
print(df.loc[(p,t,f,s,v,e,0)]['minimizer_src'])

In [ ]:
# Exclude the QPU runner-up.
df_metrics = df_metrics.drop((p,t,f,s,v,e))

### The best entry with 100% convergence (prob=0.999)

In [ ]:
df_metrics_prob100 = get_metrics(df_merged, delta=0.1, prob=0.999, which_fun_key='fun_exact', which_time_key='total_q_shots')
df_metrics_prob100 = df_metrics_prob100[(df_metrics_prob100['s']==1) & (df_metrics_prob100['num_repetitions']>1)]

In [ ]:
idxmin_prob100 = df_metrics_prob100['T_ave'].idxmin()
idxmin_prob100

In [ ]:
df_metrics_prob100.loc[[idxmin_prob100]]

In [ ]:
df.loc[idxmin_prob100]

In [ ]:
# Platform, Team, Experiment, minimizer Function, number of Samples, number of function eValuations, Repetition
(p,t,f,s,v,e) = idxmin_prob100
# Plot.
plot(df.loc[[(p,t,f,s,v,e,k) for k in range(len(df.loc[(p,t,f,s,v,e)]))]],
     xmax=30, xstep=1, ymin=-3.00, ymax=-0.00+0.01, legend_loc='upper right')

In [ ]:
# Exclude the best entry with 100% convergence.
df_metrics_prob100 = df_metrics_prob100.drop(idxmin_prob100)

### The most accurate entry (also the runner-up with 100% convergence!)

In [ ]:
df_metrics_delta0 = get_metrics(df_merged, delta=0.01, prob=0.999, which_fun_key='fun_exact', which_time_key='total_q_shots')
df_metrics_delta0 = df_metrics_delta0[(df_metrics_delta0['num_repetitions']>1)]

In [ ]:
idxmin_delta0 = df_metrics_delta0['T_ave'].idxmin()
idxmin_delta0

In [ ]:
# Also, the runner-up entry with 100% convergence!
idxmin_prob100 = df_metrics_prob100['T_ave'].idxmin()
idxmin_prob100

In [ ]:
df_metrics_delta0.loc[[idxmin_delta0]]

In [ ]:
df.loc[idxmin_delta0]

In [ ]:
# Platform, Team, Experiment, minimizer Function, number of Samples, number of function eValuations, Repetition
(p,t,f,s,v,e) = idxmin_delta0
# Plot.
plot(df.loc[[(p,t,f,s,v,e,k) for k in range(len(df.loc[(p,t,f,s,v,e)]))]],
     xmax=9, xstep=1, ymin=-3.00, ymax=-0.00+0.01, legend_loc='upper right')

In [ ]:
# Print the minimizer source.
print(df.loc[(p,t,f,s,v,e,0)]['minimizer_src'])

### Plot convergence

In [ ]:
# # Plot all.
# plot(df, legend_loc='center')

In [ ]:
# Plot QPU only.
plot(df, platform_set=[platform_qpu], markersize_divisor=10,
     xmin=0, xmax=34+0.01, xstep=1, ymin=-2.80, ymax=-1.80-0.01, ystep=0.05, legend_loc='lower left')

In [ ]:
# Plot COBYLA only.
plot(df, minimizer_method_set=['my_cobyla'], sample_number_set=[50], markersize_divisor=5,
     xmin=5, xmax=32+0.01, xstep=1, ymin=-2.89, ymax=-1.79+0.01, ystep=0.05, legend_loc='upper right')

### Plot execution metrics

In [ ]:
# plot_metric(df)

In [ ]:
# plot_metric(df, metric='total_q_shots')